# Chapter 02-04: 레이어와 활성화 함수

## 학습 목표
- 주요 활성화 함수의 수식과 특성을 이해한다
- Dense, Dropout, BatchNormalization 레이어를 올바르게 사용한다
- 가중치 초기화 방법과 정규화 기법을 적용한다

## 목차
1. 수학적 기초 — 활성화 함수
2. 활성화 함수 시각화
3. Dense와 Dropout 사용법
4. BatchNormalization vs LayerNormalization
5. 가중치 초기화 비교
6. L1/L2 정규화

---
### 🐣 초등학생을 위한 활성화 함수 친절 설명!

#### ❓ 왜 활성화 함수가 필요해요?

활성화 함수가 없으면, 층을 아무리 많이 쌓아도
결국 **단순한 직선(1차 방정식)** 밖에 표현 못해요!

> 💡 **비유**: 아무리 직선을 많이 이어 붙여도 직선이에요.
> 활성화 함수는 '구부러짐'을 추가해서 복잡한 형태를 만들어줘요!

#### 🎯 주요 활성화 함수 — 언제 무엇을 쓰나요?

| 함수 | 핵심 특성 | 언제 써요? | 비유 |
|------|-----------|-----------|------|
| **ReLU** | 음수→0, 양수→그대로 | 은닉층 기본 선택 | 전등 스위치: 어두우면 끔 💡 |
| **Sigmoid** | 0~1 사이 출력 | 이진 분류 출력층 | 확률 계산기 🎲 |
| **Softmax** | 합이 1인 확률 분포 | 다중 분류 출력층 | 여러 선택지 투표 🗳️ |
| **Tanh** | -1~1 사이 출력 | RNN 내부 | 중심이 0인 Sigmoid 📊 |
| **ELU** | 음수도 완전히 0 아님 | 더 깊은 네트워크 | ReLU의 개선판 🔧 |

#### 🛑 ReLU의 **죽은 뉴런(Dying ReLU)** 문제

ReLU는 입력이 음수면 항상 0을 출력해요.
학습 중에 어떤 뉴런이 **항상 음수 입력만 받으면** 영원히 0!
그 뉴런은 '죽은 것'처럼 학습에 기여를 못해요.

> **해결책**: Leaky ReLU (음수에도 아주 작은 기울기 유지)
> 또는 ELU (음수에서 부드럽게 감소)

#### 🧹 Dropout이 뭐예요?

학습 중에 일부 뉴런을 **무작위로 끄는** 기법이에요!

> 💡 **비유**: 팀플에서 일부러 몇 명을 뺐다가 넣었다가!
> 특정 사람에만 의존하지 않고 팀 전체가 골고루 역할을 익히게 해요.
> → 과적합(암기만 하는 현상) 방지!

#### ⚖️ BatchNorm이 뭐예요?

층을 통과할 때마다 값의 분포가 크게 달라지는 문제(내부 공변량 변화)를 해결해요.

> 💡 **비유**: 각 수업 후 '점수를 표준화'해서 항상 비슷한 분포로 만들기!
> → 학습이 더 안정적이고 빨라져요!

In [ ]:
import sys
sys.path.append('..')  # 상위 폴더의 utils 모듈 접근을 위한 경로 추가

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print("TensorFlow 버전:", tf.__version__)

## 수학적 기초 — 활성화 함수

| 함수 | 수식 | 범위 |
|------|------|------|
| ReLU | $f(x) = \max(0, x)$ | $[0, +\infty)$ |
| Sigmoid | $\sigma(x) = \frac{1}{1+e^{-x}}$ | $(0, 1)$ |
| Tanh | $\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$ | $(-1, 1)$ |
| Softmax | $\sigma_i = \frac{e^{x_i}}{\sum_j e^{x_j}}$ | $(0,1)$, 합=1 |
| ELU | $f(x) = x$ if $x>0$ else $\alpha(e^x-1)$ | $(-\alpha, +\infty)$ |

**BatchNorm**: $\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$

In [ ]:
# ── 활성화 함수 시각화 ────────────────────────────────────────────────────
# utils.plot_helpers 모듈이 있으면 사용, 없으면 직접 구현
try:
    from utils.plot_helpers import plot_activation_functions
    plot_activation_functions()
except ImportError:
    # utils 모듈이 없는 경우 직접 구현
    x = np.linspace(-4, 4, 300)  # -4 ~ 4 범위의 300개 점

    # 각 활성화 함수 계산
    relu    = np.maximum(0, x)                                      # ReLU
    sigmoid = 1 / (1 + np.exp(-x))                                  # Sigmoid
    tanh    = np.tanh(x)                                            # Tanh
    elu     = np.where(x > 0, x, 1.0 * (np.exp(x) - 1))           # ELU (alpha=1)
    leaky   = np.where(x > 0, x, 0.01 * x)                        # Leaky ReLU

    fig, axes = plt.subplots(1, 5, figsize=(18, 4))
    configs = [
        ('ReLU',       relu,    'steelblue'),
        ('Sigmoid',    sigmoid, 'orangered'),
        ('Tanh',       tanh,    'green'),
        ('ELU',        elu,     'purple'),
        ('Leaky ReLU', leaky,   'brown'),
    ]

    for ax, (name, y_vals, color) in zip(axes, configs):
        ax.plot(x, y_vals, color=color, linewidth=2.5)
        ax.axhline(0, color='k', linewidth=0.8, linestyle='--')
        ax.axvline(0, color='k', linewidth=0.8, linestyle='--')
        ax.set_title(name, fontsize=12)
        ax.set_xlabel('x')
        ax.grid(True, alpha=0.3)

    plt.suptitle('주요 활성화 함수 비교', fontsize=14)
    plt.tight_layout()
    plt.show()
    print("활성화 함수 시각화 완료")

## 3. Dense와 Dropout 사용법

Dropout은 훈련 시(`training=True`)에만 뉴런을 무작위로 비활성화한다.
추론 시(`training=False`)에는 모든 뉴런을 사용하되, 출력에 `(1 - drop_rate)`를 곱하지 않는다.

In [ ]:
# ── Dropout 훈련/추론 모드 비교 ──────────────────────────────────────────
dropout_layer = tf.keras.layers.Dropout(rate=0.5)  # 50% 뉴런을 비활성화

tf.random.set_seed(42)
x_demo = tf.ones((1, 10))  # 모두 1인 입력 (변화를 쉽게 관찰하기 위함)

# 훈련 모드: 일부 값이 0으로 설정되고 나머지는 1/(1-rate)로 스케일됨
out_train = dropout_layer(x_demo, training=True)
print("훈련 모드 출력:", out_train.numpy())

# 추론 모드: 드롭아웃 없이 그대로 통과
out_infer = dropout_layer(x_demo, training=False)
print("추론 모드 출력:", out_infer.numpy())

# ── Dense 레이어 활성화 함수별 출력 분포 ────────────────────────────────
print("\n── 활성화 함수별 출력 분포 ──")
x_rand = tf.random.normal((1000, 64))  # 정규분포 입력
for act_name in ['relu', 'sigmoid', 'tanh', 'elu']:
    dense = tf.keras.layers.Dense(32, activation=act_name)
    out   = dense(x_rand)
    print(f"{act_name:10s}: min={out.numpy().min():.3f}, "
          f"max={out.numpy().max():.3f}, "
          f"mean={out.numpy().mean():.3f}")

## 4. BatchNormalization vs LayerNormalization

| 항목 | BatchNorm | LayerNorm |
|------|-----------|----------|
| 정규화 축 | 배치 방향 (샘플 간) | 특징 방향 (채널/차원) |
| 배치 크기 의존 | 있음 (작으면 불안정) | 없음 |
| 주 사용처 | CNN, 대형 배치 | RNN, Transformer |
| 추론 시 | 이동 평균 사용 | 동일한 방식 |

In [ ]:
# ── BatchNormalization vs LayerNormalization 비교 ─────────────────────────

# 정규화를 포함한 모델 구성 비교
def build_bn_model():
    """BatchNormalization을 사용한 모델"""
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128),                          # 활성화 함수 없이 Dense
        tf.keras.layers.BatchNormalization(),                # 배치 정규화 후 활성화
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dense(64),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ], name='bn_model')

def build_ln_model():
    """LayerNormalization을 사용한 모델"""
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128),
        tf.keras.layers.LayerNormalization(),                # 레이어 정규화
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dense(64),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ], name='ln_model')

bn_model = build_bn_model()
ln_model = build_ln_model()

print("BatchNorm 모델 파라미터:", bn_model.count_params())
print("LayerNorm 모델 파라미터:", ln_model.count_params())

# BatchNorm 레이어의 내부 파라미터 확인
bn_layer = tf.keras.layers.BatchNormalization()
_ = bn_layer(tf.zeros((4, 32)))  # 빌드를 위해 더미 데이터 통과
print("\nBatchNorm 파라미터 목록:")
for w in bn_layer.weights:
    print(f"  {w.name:40s} shape={w.shape}")

## 5. 가중치 초기화 비교

| 초기화 방법 | 수식 | 권장 활성화 함수 |
|------------|------|----------------|
| Glorot (Xavier) | $\text{Var}(W) = \frac{2}{n_{in} + n_{out}}$ | Sigmoid, Tanh |
| He (Kaiming) | $\text{Var}(W) = \frac{2}{n_{in}}$ | ReLU, ELU |
| LeCun | $\text{Var}(W) = \frac{1}{n_{in}}$ | SELU |

In [ ]:
# ── 초기화 방법별 가중치 분포 시각화 ─────────────────────────────────────
initializers = {
    'glorot_uniform': tf.keras.initializers.GlorotUniform(seed=42),  # Xavier 균등분포
    'he_normal':      tf.keras.initializers.HeNormal(seed=42),       # Kaiming 정규분포
    'lecun_normal':   tf.keras.initializers.LecunNormal(seed=42),    # LeCun 정규분포
}

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
shape = (784, 128)  # Dense(784→128)의 가중치 shape

for ax, (name, init) in zip(axes, initializers.items()):
    weights = init(shape=shape).numpy().flatten()  # 초기 가중치 생성 후 1D로 펼침
    ax.hist(weights, bins=60, color='steelblue', alpha=0.8, edgecolor='white')
    ax.set_title(f'{name}\nstd={weights.std():.4f}', fontsize=11)
    ax.set_xlabel('가중치 값')
    ax.set_ylabel('빈도')
    ax.grid(True, alpha=0.3)

plt.suptitle('가중치 초기화 방법별 분포 비교', fontsize=14)
plt.tight_layout()
plt.show()

# 초기화 방법을 Dense 레이어에 적용하는 예시
layer_glorot = tf.keras.layers.Dense(128, activation='relu',
                                     kernel_initializer='glorot_uniform')  # 기본값
layer_he     = tf.keras.layers.Dense(128, activation='relu',
                                     kernel_initializer='he_normal')        # ReLU 계열 권장
print("He Normal 초기화 레이어 생성 완료 (ReLU와 함께 권장)")

## 6. L1/L2 정규화

정규화는 손실함수에 패널티 항을 추가하여 가중치가 지나치게 커지는 것을 방지한다.

- **L1 정규화**: $\mathcal{L}_{reg} = \lambda \sum |w_i|$ → 희소 가중치 유도
- **L2 정규화**: $\mathcal{L}_{reg} = \lambda \sum w_i^2$ → 가중치 축소 (Weight Decay)
- **L1+L2**: 두 가지를 동시에 적용

In [ ]:
# ── kernel_regularizer로 L1/L2 정규화 적용 ──────────────────────────────

lambda_val = 1e-4  # 정규화 강도 (클수록 강한 규제)

# 정규화 방법별 레이어 생성
layer_l2 = tf.keras.layers.Dense(
    64, activation='relu',
    kernel_regularizer=tf.keras.regularizers.L2(lambda_val),   # L2 정규화
    name='l2_dense'
)

layer_l1 = tf.keras.layers.Dense(
    64, activation='relu',
    kernel_regularizer=tf.keras.regularizers.L1(lambda_val),   # L1 정규화
    name='l1_dense'
)

layer_l1l2 = tf.keras.layers.Dense(
    64, activation='relu',
    kernel_regularizer=tf.keras.regularizers.L1L2(l1=lambda_val, l2=lambda_val),  # L1+L2
    name='l1l2_dense'
)

# 정규화 포함 모델 예시
reg_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(
        128, activation='relu',
        kernel_regularizer=tf.keras.regularizers.L2(1e-4),   # 은닉층에 L2 적용
        kernel_initializer='he_normal'                         # ReLU에 맞는 초기화
    ),
    tf.keras.layers.BatchNormalization(),                      # 배치 정규화
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')
], name='regularized_model')

reg_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
reg_model.summary()
print("\n정규화 모델 컴파일 완료")

## 정리

| 기법 | 목적 | 적용 위치 |
|------|------|----------|
| Dropout | 과적합 방지 (랜덤 비활성화) | 은닉층 이후 |
| BatchNorm | 내부 공변량 변화 감소 | Dense/Conv 이후, 활성화 이전 |
| LayerNorm | 배치 크기 독립 정규화 | RNN, Transformer 내부 |
| L2 정규화 | 가중치 크기 제한 | kernel_regularizer |
| He 초기화 | ReLU 뉴런 소멸 방지 | ReLU 계열 레이어 |
| Glorot 초기화 | 신호 분산 유지 | Sigmoid/Tanh 레이어 |

**다음**: practice/ex01_build_your_first_model.ipynb — 3가지 API 통합 실습